# Utils

In [2]:
# default_exp utils
# export
import pandas as pd
from pathlib import Path

## Get Data Functions
> wrapper functions to load dataframes more convieniently

In [4]:
# export 
def get_dataframe(fn = 'dataset_training_477.csv', 
                  DATA_DIR=Path('/media/ScaleOut/vahldiek/MRI/SIJ'), 
                  preprocessed_images=True, 
                  grid_view=False, 
                  denoised = False):
    "Returns the dataframe with correct filenames for MRI-SpA project"
    
    df = pd.read_csv(DATA_DIR/fn)
    df = df[df.has_TIRM == 1]
    df = df[df.has_T1 == 1]
    df = df.rename(columns={'path_TIRM': 'TIRM', 
                            'path_T1': 'T1', 
                            'ASAS_compatible_changes': 'asas_compatible_changes'})

    if preprocessed_images: 
        df['TIRM'] = ['01_preprocessed/' + fn + '_TIRM.nii.gz' for fn in df.patient_ID]
        df['T1'] = ['01_preprocessed/' + fn + '_T1.nii.gz' for fn in df.patient_ID]
    
    if denoised:
        df['TIRM'] = ['05_denoised/' + fn + '_TIRM.nii.gz' for fn in df.patient_ID]
        df['T1'] = ['05_denoised/' + fn + '_T1.nii.gz' for fn in df.patient_ID]
        
    if grid_view:
        df['TIRM'] = ['03_grid_view/' + fn + '_TIRM.png' for fn in df.patient_ID]
        df['T1'] = ['03_grid_view/' + fn + '_T1.png' for fn in df.patient_ID]
        
    df['asas_compatible_changes'] = ['asas_compatible_changes' if x == '1' else 'no_asas_compatible_changes' for x in df.asas_compatible_changes]     
    df['active_changes'] = ['active_changes' if x == 1 else 'no_active_changes' for x in df.active_changes]
    df['structural_changes'] = ['structural_changes' if x == 1 else 'no_structural_changes' for x in df.structural_changes]
    combined_label = []
    for act, struc in zip(df.active_changes, df.structural_changes):
        if 'no' in act: act = ''
        if 'no' in struc: struc = ''
        delim = ';' if act != '' and struc != '' else ''
        combined_label.append(act + delim + struc)
    df['combined_label'] = combined_label
    
    return df

In [5]:
df = get_dataframe()

In [6]:
# export 
def get_test_dataframe(DATA_DIR=Path('/media/ScaleOut/vahldiek/MRI/SIJ'), 
                       preprocessed_images=True, 
                       grid_view=False, 
                       denoised=False):
    "Returns the test dataframe with correct filenames for MRI-SpA project"
    
    df = pd.read_csv(DATA_DIR/'ASAS_holdout_130_with_asas.csv')
    df = df[df.has_TIRM == 1]
    df = df[df.has_T1 == 1]
    df = df.rename(columns={'struct_changes': 'structural_changes', 
                            'path_TIRM': 'TIRM', 
                            'path_T1': 'T1', 
                            'ASAS_compatible_changes': 'asas_compatible_changes'})

    if preprocessed_images: 
        df['TIRM'] = ['01_preprocessed/' + fn + '_TIRM.nii.gz' for fn in df.patient]
        df['T1'] = ['01_preprocessed/' + fn + '_T1.nii.gz' for fn in df.patient]
    
    if denoised:
        df['TIRM'] = ['05_denoised/' + fn + '_TIRM.nii.gz' for fn in df.patient]
        df['T1'] = ['05_denoised/' + fn + '_T1.nii.gz' for fn in df.patient]
            
    if grid_view:
        df['TIRM'] = ['03_grid_view/' + fn + '_TIRM.png' for fn in df.patient]
        df['T1'] = ['03_grid_view/' + fn + '_T1.png' for fn in df.patient]

    df['active_changes'] = ['active_changes' if x == 1 else 'no_active_changes' for x in df.active_changes]    
    df['structural_changes'] = ['structural_changes' if x == 1 else 'no_structural_changes' for x in df.structural_changes]
    df['asas_compatible_changes'] = ['asas_compatible_changes' if x == 1 else 'no_asas_compatible_changes' for x in df.asas_compatible_changes]     

    combined_label = []
    for act, struc in zip(df.active_changes, df.structural_changes):
        if 'no' in act: act = ''
        if 'no' in struc: struc = ''
        delim = ';' if act != '' and struc != '' else ''
        combined_label.append(act + delim + struc)
    df['combined_label'] = combined_label
    df = df[-df.patient.isin(['Case070', 'Case043', 'Case134'])] # need to be excluded because of missing data
    return df

In [7]:
df = get_test_dataframe()

### Metrics
Add more metrics to fastai's ClassificationInterpretation class

In [8]:
# export 
import sklearn.metrics as metrics
import torch
from fastcore.all import *
from fastai.metrics import *
from fastai.interpret import ClassificationInterpretation
from fastai.torch_core import flatten_check

In [10]:
# export
@patch 
def plot_roc(self:ClassificationInterpretation):
    "Plot a ROC curve with AUC from decoded preds and targets"
    # calculate the fpr and tpr for all thresholds of the classification
    d,t = flatten_check(self.preds[:, 1], self.targs)
    d = d.numpy()
    t = t.numpy()
    fpr, tpr, threshold = metrics.roc_curve(t, d)
    roc_auc = metrics.auc(fpr, tpr)

    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [11]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_preprocessing.ipynb.
Converted 02_augment.ipynb.
Converted 03_noise-reduction.ipynb.
Converted 04_classification_models.ipynb.
Converted 05_classification.ipynb.
Converted 06_evaluation.ipynb.
Converted 07_descriptive.ipynb.
Converted 99_widgets.cam.ipynb.
Converted index.ipynb.
